In [ ]:
import time
import torch
import random
import numpy as np
import PIL.Image
from src.ichimatsu import get_pipe

def torch_fix_seed(seed=42):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True

pretrained_model_name_or_path = "./model/"
lora_checkpoint_path = ""
vae_path = ""

alpha = 1.0
pipe = get_pipe(pretrained_model_name_or_path, lora_checkpoint_path, vae_path=vae_path, torch_dtype=torch.float16, alpha=alpha)

In [ ]:
prompts = "best quality, 1girl"
negative_prompts = "nsfw, low quality, worst quality"
strength = 0.5
guidance_scale = 12.5
scale = 1

use_search_mean_noise = False

# TrueがIchimatsu pipeline, Falseはdiffusersのi2i
use_ichimatsu_pipeline = True

import glob

mov_list = glob.glob("./input/*.png")

ut = time.time()
exe_name = "./result/" +"result_"+str(ut)

for i in range(len(mov_list)):
  print("\r", i+1, "/", len(mov_list), end="                    ")

  input_path = mov_list[i]

  seed = 3
  if(seed >= 0):
    torch_fix_seed(seed=seed)

  image = PIL.Image.open(input_path).convert('RGB')
  image_result = pipe.__call__(\
      prompt=prompts,\
      image=image, \
      strength=strength, \
      guidance_scale=guidance_scale, \
      negative_prompt=negative_prompts, \
      use_ichimatsu_pipeline = use_ichimatsu_pipeline, \
  )[0][0]
  outputFileName = exe_name+"_"+str(i).zfill(4)+"_final_result"+".png"
  image_result.resize(image.size, PIL.Image.LANCZOS).save(outputFileName)
print("DONE")